![CONUS_Radar_2019_05_21_1200Z.png](Jupyter_Images/CONUS_Radar_2019_05_21_1200Z.png)
CONUS_Radar_2019_05_21_1200Z.png

In [1]:
from netCDF4 import Dataset
from netCDF4 import num2date
import glob, os

import matplotlib.pyplot as plt

import cartopy.feature as cfeature

import cartopy.crs as ccrs

import numpy as np

from matplotlib import patheffects

from siphon.catalog import TDSCatalog

## THREDDS Catalog Data Access

In [2]:
from metpy.plots import ctables
mycmap = ctables.registry.get_colortable("NWSReflectivity")

/Users/chowdahead/miniconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/chowdahead/miniconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
from siphon.catalog import TDSCatalog
top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
ref = top_cat.catalog_refs['Radar Data']

new_cat = ref.follow()
model = new_cat.catalog_refs[3]
model

Unidata NEXRAD Composites (GINI)

In [8]:
def Map(Year,Month,Day,Hour,Min):
    
    from siphon.catalog import TDSCatalog
    top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
    ref = top_cat.catalog_refs['Radar Data']

    new_cat = ref.follow()
    model = new_cat.catalog_refs[4]
    gfs_cat = model.follow()

    ds = gfs_cat.datasets[0]
    subset = ds.subset()
    query_data = subset.query()
        #query_data.lonlat_box(west=-130, east=-50, south=10, north=60)
    from datetime import datetime, timedelta
    #now = datetime.utcnow()
    now = datetime(Year,Month,Day,Hour,Min)
    #print(now)
        # Allow for NetCDF files
    query_data.accept('netcdf4')
    #query_data.lonlat_box(west=-129, east=-64, south=21, north=48)
    query_data.time(now)
    query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

    # Finally attempt to access the data
    data = subset.get_data(query_data)
    #print(list(data.variables))

    time = data.variables['time']
    lon = data.variables['lon']
    lat = data.variables['lat']
    dtime = num2date(time[:],time.units)
    X = data.variables['x'][:]
    Y = data.variables['y'][:]

    Time = dtime[0].strftime('%Y-%m-%d %H:%MZ')
    print(Time)
    reflec_data = data.variables['Base_reflectivity_surface_layer']
    reflec = data.variables['Base_reflectivity_surface_layer'][:]

    
    
    fig = plt.figure(figsize=(17., 11.))


    # Add state and country boundaries to plot
    states_boundaries = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

    country_borders = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries',scale='50m', facecolor='none')

    plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

    #plotcrs = ccrs.LambertConformal(central_longitude=cent_lon)
    #pl = ccrs.PlateCarree(central_longitude=cent_lon)


    ax = plt.subplot(111,projection=ccrs.LambertConformal())

    ax.add_feature(states_boundaries,edgecolor='b',linewidth=1)
    ax.add_feature(country_borders,edgecolor='b',linewidth=1)

    # Lat/Lon Extents [lon0,lon1,lat0,lat1]
    extent = [-130., -70, 20., 60.]
    ax.set_extent(extent,ccrs.PlateCarree()) 

    #ax.imshow(reflec[0,:,:],cmap='nipy_spectral',interpolation='nearest',
    #          extent=(-129.,-64,22,48),transform=ccrs.PlateCarree()) #transform=ccrs.PlateCarree()

    #ax.contourf(lon[:],lat[:],reflec[0,:,:],cmap='nipy_spectral',transform=ccrs.PlateCarree())
    RADAR_levs = np.arange(0,75,1)
    cs = ax.pcolormesh(lon[:], lat[:], reflec[0,:,:], cmap=mycmap,transform=ccrs.PlateCarree(),vmin=0, vmax=75)
    cbar = plt.colorbar(cs, orientation='vertical',pad=0.005,aspect=50)
    text_time = ax.text(.995, 0.01, 
            Time,
            horizontalalignment='right', transform=ax.transAxes,
            color='white', fontsize=15, weight='bold')

    text_time2 = ax.text(0.005, 0.01, 
                "NWS Radar (dbz)",
                horizontalalignment='left', transform=ax.transAxes,
                color='white', fontsize=15, weight='bold')

    outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
    text_time.set_path_effects(outline_effect)
    text_time2.set_path_effects(outline_effect)

    # Plot Title
    #plt.title('NWS Radar (dbz)',loc='left',fontsize=16)
    #plt.title(Time,loc='right',fontsize=16)
    #plt.show()
    #plt.savefig("/Users/chowdahead/Desktop/Radar/Radar_plot_Lamb_data_Lamb_COLORMESH.png",
    #           bbox_inches='tight')
    plt.savefig("/Users/chowdahead/Desktop/Weather_Blog/Radar/CONUS_Radar_"\
                        +Time[0:10].replace("-","_")+"_"+Time[-6:].replace(":","")+".png",bbox_inches="tight")
    plt.close(fig)




In [9]:
Map(2019,7,12,12,0)

2019-07-12 12:00Z
